In [ ]:
!pip install hvplot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy as np
import hvplot.pandas
import holoviews as hv
import seaborn as sns


data = pd.read_csv('https://raw.githubusercontent.com/ChrisWalshaw/DataViz/master/Data/001207067/OutletDailyCustomers.csv',index_col=0)
data2 = pd.read_csv('https://raw.githubusercontent.com/ChrisWalshaw/DataViz/master/Data/001207067/OutletMarketing.csv', index_col=0)
data3 = pd.read_csv('https://raw.githubusercontent.com/ChrisWalshaw/DataViz/master/Data/001207067/OutletOverheads.csv', index_col=0)
data4 = pd.read_csv('https://raw.githubusercontent.com/ChrisWalshaw/DataViz/master/Data/001207067/OutletSize.csv', index_col=0)
data5 = pd.read_csv('https://raw.githubusercontent.com/ChrisWalshaw/DataViz/master/Data/001207067/OutletStaff.csv', index_col=0)
summary_data = pd.DataFrame(index=data.columns)
summary_data['Marketing'] = data2.values
summary_data['Overheads'] = data3.values
summary_data['TotalCustomers'] = data.sum().values
summary_data['Size'] = data4.values
summary_data['Staff'] = data5.values


# Pie chart for the volume of daily visitors

In [ ]:
data = data.reindex(data.sum().sort_values(ascending=False).index, axis=1)
explodeList = []
selected = []
high_volume = []
medium_volume = []
low_volume = []
columns = data.columns
data['Others'] = [0] * len(data.index)
for name in columns:
    total_sales = data[name].sum()
    if total_sales > 200000:
        selected.append(name)
        explodeList.append(0)
        if(total_sales>600000):
          high_volume.append(name)
        else:
          medium_volume.append(name)
    else:
        data['Others'] += data[name]
        low_volume.append(name)
selected.append('Others')
explodeList.append(0.05)
plt.figure(figsize=(8, 8))
plt.pie(data[selected].sum(), labels=selected, autopct='%1.1f%%', startangle=90, explode=explodeList)
plt.title('Total Customer Visits', fontsize=20)
plt.legend(loc=2)
plt.show()

data = data.drop('Others', axis=1)


# Radar Plots for Medium Volume Outlets




In [ ]:
summary_data = summary_data.reindex(columns=['Marketing', 'TotalCustomers', 'Size', 'Staff','Overheads'])

normalised_data = summary_data / summary_data.loc[medium_volume].max()
n_attributes = len(normalised_data.columns)
angles = [n / float(n_attributes) * 2 * np.pi for n in range(n_attributes + 1)]
plt.figure(figsize=(12, 12))
counter = 1
for name in medium_volume:
    values = normalised_data.loc[[name]].values.flatten().tolist()
    values += values[:1]
    sub = plt.subplot(4, 4, counter, polar=True)
    sub.plot(angles, values)
    sub.fill(angles, values, alpha=0.1)
    sub.set_ylim(ymax=1.05)
    sub.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    sub.set_xticks(angles[0:-1])
    sub.set_xticklabels(normalised_data.columns, fontsize=8)
    sub.set_title('Outlet ' + name, fontsize=12, loc='left')
    counter += 1
plt.tight_layout()
plt.show()

# Radar Plots for High Volume Outlets


In [ ]:
normalised_data = summary_data / summary_data.loc[high_volume].max()
n_attributes = len(normalised_data.columns)
angles = [n / float(n_attributes) * 2 * np.pi for n in range(n_attributes + 1)]
plt.figure(figsize=(12, 12))
counter = 1
for name in high_volume:
    values = normalised_data.loc[[name]].values.flatten().tolist()
    values += values[:1]
    sub = plt.subplot(4, 4, counter, polar=True)
    sub.plot(angles, values)
    sub.fill(angles, values, alpha=0.1)
    sub.set_ylim(ymax=1.05)
    sub.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    sub.set_xticks(angles[0:-1])
    sub.set_xticklabels(normalised_data.columns, fontsize=8)
    sub.set_title('Outlet ' + name, fontsize=12, loc='left')
    counter += 1
plt.tight_layout()
plt.show()

# Scatter Plot Correlation on different variables of summary data of low volume outlets


*(Marketing, Total Customers, Size, Staff and Overheads)*

In [ ]:
new_df = summary_data.drop(medium_volume+high_volume)
sns.pairplot(new_df, height=1.5, plot_kws={'s': 20})
plt.suptitle("Scatter Plot of Summary Data of Low Volume Outlets",y=1.05)
plt.show()


# HeatMap Correlation on different variables of summary data of low volume outlets


*(Marketing, Total Customers, Size, Staff and Overheads)*

In [ ]:
plt.figure(figsize=(6, 6))
corr = new_df.corr()
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(220, 20, n=200), square=True, annot=True,
                 annot_kws={"size": 8})
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.suptitle("HeatMap for Summary Data of Low Volume Outlets",y=1.05)

plt.show()


In [ ]:
counter = 1
fig = plt.figure(figsize=(16, 16))
fig.suptitle('Outlets Visitor Line Chart', fontsize=20, position=(0.5, 1.0))
for name in data.columns:
    sub = fig.add_subplot(8, 8, counter)
    sub.set_title('Outlet ' + name, fontsize=10)
    if(name in low_volume):
      sub.plot(data.index, data[name], linewidth=0.5)
    elif(name in medium_volume):
      sub.plot(data.index, data[name], linewidth=0.5, color = "green")
    else:
      sub.plot(data.index, data[name], linewidth=0.5, color = "red")
    sub.axes.get_xaxis().set_ticks([])  # remove the x ticks
    counter += 1
plt.subplots_adjust(wspace=0.7, hspace=0.7)

plt.show()

# Trendline for the High Volume Pages

In [ ]:
period = 90
rolling_average = data.rolling(window=period).mean()
print(data[high_volume].head())
plt.figure(figsize=(8, 8))
plt.plot(data[high_volume], linewidth=0.5)
# plt.gca().set_prop_cycle(None)
# plt.plot(rolling_average[selected], linewidth=2)
plt.gca().set_prop_cycle(None)
for name in high_volume:
    x = np.arange(len(data[name]))
    z = np.polyfit(x, data[name], 1)
    trend = np.poly1d(z)
    plt.plot(data.index, trend(x), linestyle='--')
    slope, intercept = z
    print("Slope of the trend line for "+name+" :", slope)

plt.ylim(ymin=0)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Volume of Visitors', fontsize=18)
plt.title('Correlated Volume of Visitors', fontsize=20)
plt.legend(high_volume, loc=2)
plt.show()

# Trendline for the Medium Volume Pages

In [ ]:

import numpy as np

period = 90
rolling_average = data.rolling(window=period).mean()


print(data[medium_volume].head())

plt.figure(figsize=(8, 8))
plt.plot(data[medium_volume], linewidth=0.5)
# plt.gca().set_prop_cycle(None)
# plt.plot(rolling_average[selected], linewidth=2)
plt.gca().set_prop_cycle(None)
for name in medium_volume:
    x = np.arange(len(data[name]))
    z = np.polyfit(x, data[name], 1)
    trend = np.poly1d(z)
    plt.plot(data.index, trend(x), linestyle='--')
    slope, intercept = z
    print("Slope of the trend line for "+name+" :", slope)
plt.ylim(ymin=0)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Volume of Visitors', fontsize=18)
plt.title('Correlated Volume of Visitors', fontsize=20)
plt.legend(medium_volume, loc=2)
plt.show()

# High Volume Outlets Correlation

In [ ]:

counter = 1
fig = plt.figure(figsize=(8, 8))
fig.suptitle('High Volume Visitor Outlets correlations', fontsize=14, position=(0.5, 1.0))
for i, name_i in enumerate(high_volume):
    for j in range(i + 1, len(high_volume)):
        name_j = high_volume[j]
        sub = fig.add_subplot(6,6, counter)
        sub.set_title(name_i + ' vs ' + name_j, fontsize=10)
        sub.scatter(data[name_i], data[name_j], s=0.5)
        counter += 1
plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(4, 4))
corr = data[high_volume].corr()
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(220, 20, n=200), square=True, annot=True,
                 annot_kws={"size": 8})
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.suptitle("HeatMap for Correlation of High Volume Visitors Outlets",y=1.05)

plt.show()

# Medium Volume Outlet Correlation

In [ ]:

counter = 1
fig = plt.figure(figsize=(8, 8))
fig.suptitle('Medium Volume Visitor Outlets correlations', fontsize=14, position=(0.5, 1.0))
for i, name_i in enumerate(medium_volume):
    for j in range(i + 1, len(medium_volume)):
        name_j = medium_volume[j]
        sub = fig.add_subplot(6,6, counter)
        sub.set_title(name_i + ' vs ' + name_j, fontsize=10)
        sub.scatter(data[name_i], data[name_j], s=0.5)
        counter += 1
plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
corr = data[medium_volume].corr()
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(220, 20, n=200), square=True, annot=True,
                 annot_kws={"size": 8})
plt.suptitle("HeatMap for Correlation of Medium Volume Visitors Outlets",y=1.05)

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

# Heatmap Correlation for SummaryData

In [ ]:

plt.figure(figsize=(8, 8))
corr = summary_data.corr()
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(220, 20, n=200), square=True, annot=True,
                 annot_kws={"size": 8})
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_yticklabels(ax.get_yticklabels(), rotation=45, horizontalalignment='right')
plt.suptitle("HeatMap for Correlation of Summary data variables",y=1.05)


plt.show()


Interactive Visualisation

In [ ]:
summary_data['BubbleSize'] = summary_data['Size'] * 3.0


plot = summary_data.hvplot.scatter(
    frame_height=500, frame_width=500,
    title='TotalCustomers vs Marketing (vs Size)',
    xlabel='Marketing (£/unit)', ylabel='Customers (£/unit)',
    alpha=0.5, padding=0.1, hover_cols='all',
    tools=['pan', 'box_zoom', 'wheel_zoom', 'undo', 'redo', 'hover', 'save', 'reset'],
    x='Marketing', y='TotalCustomers', size='Size'
)
hv.extension('bokeh')
plot

In [ ]:

import numpy as np
period = 90
rolling_average = data[medium_volume].rolling(window=period).mean()
plt.figure(figsize=(8, 8))
plt.plot(data[medium_volume], linewidth=0.5)
plt.gca().set_prop_cycle(None)
plt.plot(rolling_average[medium_volume], linewidth=2)
# plt.ylim(ymin=0)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Page Hits', fontsize=18)
plt.title('Medium Volume Page Hits\n with 14-day Rolling Average', fontsize=20)
plt.legend(medium_volume, loc=2)
plt.show()

In [ ]:
selected = high_volume
for name in selected:
    pd.plotting.autocorrelation_plot(data[name])
    # plt.xlim([0, 50]) # uncomment this line to zoom in
    plt.title('Page ' + name)
    plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
selected = high_volume

for name in selected:
    result = seasonal_decompose(data[name], model='multiplicative', period=7)
    result.plot()
    plt.suptitle('Outlet ' + name,
        position=(0.5, 1.0))
    plt.show()

In [ ]:
plot = data[high_volume].hvplot.line(
    frame_height=500, frame_width=500,
    xlabel='Date', ylabel='Hits',
    title='High Volume Outlets'
)
hv.extension('bokeh')
plot



In [ ]:
plot = data[medium_volume].hvplot.line(
    frame_height=500, frame_width=500,
    xlabel='Date', ylabel='Hits',
    title='Medium Volume Outlets'
)
hv.extension('bokeh')
plot

In [ ]:
x_min = 1400
x_max = 2500
bin_width = 15
n_bins = int((bin_width + x_max - x_min) / bin_width)
print(str(n_bins) + ' bins')
bins = [(x_min + x * (bin_width + x_max - x_min) / n_bins) for x in range(int(n_bins))]

plot = data[high_volume].hvplot.hist(
    frame_height=500, frame_width=500,
    xlabel='Units sold per day', ylabel='Frequency',
    title='High Volume Outlets',
    alpha=0.5, muted_alpha=0, muted_fill_alpha=0, muted_line_alpha=0,
    tools=['pan', 'box_zoom', 'wheel_zoom', 'undo', 'redo', 'hover', 'save', 'reset'],
    bins=bins
)
hv.extension('bokeh')
plot

In [ ]:
x_min = 400
x_max = 1500
bin_width = 18
n_bins = int((bin_width + x_max - x_min) / bin_width)
print(str(n_bins) + ' bins')
bins = [(x_min + x * (bin_width + x_max - x_min) / n_bins) for x in range(int(n_bins))]

plot = data[medium_volume].hvplot.hist(
    frame_height=500, frame_width=500,
    xlabel='Units sold per day', ylabel='Frequency',
    title='Medium Volume Outlets',
    alpha=0.5, muted_alpha=0, muted_fill_alpha=0, muted_line_alpha=0,
    tools=['pan', 'box_zoom', 'wheel_zoom', 'undo', 'redo', 'hover', 'save', 'reset'],
    bins=bins
)
hv.extension('bokeh')
plot

In [ ]:
summary_data['BubbleSize'] = summary_data['Size'] * 3.0
print(summary_data.head())


plot = summary_data.hvplot.scatter(
    frame_height=500, frame_width=500,
    title='TotalCustomers vs Marketing (vs Size)',
    xlabel='HitTotalCustomers (£/unit)', ylabel='RevMarketingenue (£/unit)',
    alpha=0.5, padding=0.1, hover_cols='all',
    tools=['pan', 'box_zoom', 'wheel_zoom', 'undo', 'redo', 'hover', 'save', 'reset'],
    x='TotalCustomers', y='Marketing', size='Size'
)
hv.extension('bokeh')
plot

In [ ]:

plt.figure(figsize=(8, 8))
# data[selected].boxplot()
plt.boxplot(data[high_volume], labels=high_volume)
plt.xlabel('Page Number', fontsize=18)
plt.ylabel('Page Hits per day', fontsize=18)
plt.title('High volume Page Hits distributions', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
# data[selected].boxplot()
plt.boxplot(data[medium_volume], labels=medium_volume)
plt.xlabel('Page Number', fontsize=18)
plt.ylabel('Page Hits per day', fontsize=18)
plt.title('High volume Page Hits distributions', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
counter = 1
x_pos = np.arange(len(summary_data.index))
print(summary_data)
for attribute in summary_data:
    sub = plt.subplot(3, 3, counter)
    sub.bar(x_pos, summary_data[attribute], align='center')
    sub.set_xticks([])
    sub.set_xticks(x_pos)
    sub.set_xticklabels(summary_data.index)
    sub.set_xlabel('Outlet', fontsize=12)
    sub.set_ylabel(attribute, fontsize=12)
    counter += 1
plt.tight_layout()
plt.show()